# Hummingbot simple backtest

Assume we quote continuously to given spread. Measure short term profitability of our fills to future price, 1 000 
trades later, for different spread settings. 

We use [crypto-lake.com](https://crypto-lake.com/#data) example market data, FTRB-USDT market on Ascendex.

In [1]:
import datetime
import math

import numpy as np
import pandas as pd
import cufflinks as cf

import lakeapi

cf.go_offline()
lakeapi.use_sample_data(anonymous_access=True)

In [2]:
# Parameters
symbol = 'FTRB-USDT'
exchange = 'ASCENDEX'
tick_size_decimals = 6
fees_bps = 16 # ascendex vip2, maker order, altcoin

start = datetime.datetime(2022, 9, 1)
end = datetime.datetime(2022, 10, 15)

## Data

In [3]:
def load_data(table: str):
	print('Loading', table)
	return lakeapi.load_data(
		table = table,
		start = start,
		end = end,
		symbols = [symbol],
		exchanges = [exchange],
		drop_partition_cols = True,
	).sort_values('received_time')

# Load l1 data = top of the order book
l1 = load_data('level_1')
# Load trades
trades = load_data('trades')

Loading level_1


  0%|          | 0/44 [00:00<?, ?it/s]

Loading trades


  0%|          | 0/44 [00:00<?, ?it/s]

In [4]:
# Merge trades and l1 depth data
df = pd.merge_asof(
	left = trades.rename(columns = {'origin_time': 'trade_origin_time', 'received_time': 'trade_received_time'}),
	right = l1.rename(columns = {'received_time': 'depth_received_time'}),
	left_on = 'trade_received_time',
	right_on = 'depth_received_time',
	tolerance = pd.Timedelta(minutes = 120),
)
df['mid'] = (df.bid_0_price + df.ask_0_price) / 2
df.head(3)

,side,quantity,price,trade_id,trade_origin_time,trade_received_time,depth_received_time,bid_0_price,ask_0_price,bid_0_size,ask_0_size,mid
0,sell,6000.0,0.010031,108086470184688960,2022-09-01 14:27:52.234999808,2022-09-01 14:27:52.275970048,2022-09-01 14:27:43.411748864,0.010029,0.010033,46720.0,13529.0,0.010031
1,buy,5000.0,0.010031,108086470184713728,2022-09-01 14:28:40.232999936,2022-09-01 14:28:40.362131712,2022-09-01 14:28:18.765389568,0.010029,0.010033,41836.0,18642.0,0.010031
2,buy,6000.0,0.010031,108086470184732864,2022-09-01 14:29:18.233999872,2022-09-01 14:29:18.543058432,2022-09-01 14:28:18.765389568,0.010029,0.010033,41836.0,18642.0,0.010031


## Backtest

In [7]:
results = []
# Show details of 100bps spread variant in the plot below
spread_to_plot = 100
df_to_plot = None
for spread_bps in range(20, 200, 10):
	df['spread_bps'] = spread_bps
	df['future_price'] = df.mid.shift(-1000)
	df['quote_bid'] = round(df.bid_0_price * (1 - df.spread_bps * 0.0001), tick_size_decimals)
	df['quote_ask'] = round(df.ask_0_price * (1 + df.spread_bps * 0.0001), tick_size_decimals)
	df['fill_sign'] = 0
	df.loc[df.quote_bid > df.price, 'fill_sign'] = 1
	df.loc[df.quote_ask < df.price, 'fill_sign'] = -1
	df['gross_profitability_short_bps'] = 10_000 * (df.future_price - df.mid) / df.mid * df.fill_sign 
	df['net_profitability_short_bps'] = (df['gross_profitability_short_bps'] - fees_bps) * df.fill_sign.abs()
	fill_count = df.fill_sign.abs().sum()
	results.append((spread_bps, fill_count, df.gross_profitability_short_bps.sum() / fill_count))
	if spread_to_plot == spread_bps:
		df_to_plot = df.copy()

result_df = pd.DataFrame(results, columns = ['spread_bps', 'fill_count', 'gross_profitability_short_bps'])
result_df['net_profitability_short_bps'] = result_df.gross_profitability_short_bps - fees_bps
result_df['total_gross_profit_bps'] = result_df.gross_profitability_short_bps * result_df.fill_count
result_df['total_net_profit_bps'] = result_df.net_profitability_short_bps * result_df.fill_count
result_df

,spread_bps,fill_count,gross_profitability_short_bps,net_profitability_short_bps,total_gross_profit_bps,total_net_profit_bps
0,20,208,-91.370109,-107.370109,-19004.982770,-22332.982770
1,30,157,-61.212598,-77.212598,-9610.377964,-12122.377964
2,40,125,-21.438402,-37.438402,-2679.800274,-4679.800274
3,50,108,-13.592648,-29.592648,-1468.005971,-3196.005971
4,60,93,-7.420503,-23.420503,-690.106791,-2178.106791
5,70,78,-13.677044,-29.677044,-1066.809422,-2314.809422
6,80,72,-10.531684,-26.531684,-758.281268,-1910.281268
7,90,61,29.935264,13.935264,1826.051076,850.051076
8,100,55,43.380546,27.380546,2385.930014,1505.930014
9,110,48,83.154713,67.154713,3991.426237,3223.426237


### Plots

In [12]:
# Plot trades and the spread
df_to_plot.set_index('trade_received_time')[['bid_0_price', 'ask_0_price', 'price']][:3000:10].iplot(yTitle = 'price')

## More data

What data are available in the anonymous sample repository. Uncomment to explore:

In [10]:
# for table in ('level_1', 'trades', 'book', 'candles'):
# 	available_data = pd.DataFrame(lakeapi.list_data(table = table))
# 	print(table)
# 	display(available_data[['exchange', 'symbol', 'dt']].groupby(['exchange', 'symbol']).aggregate({'dt': ['first', 'last']}))

## Conclusion

Average reward on FTRB-USDT is currently 358% p.a. for average spread of 40bps. In this model scenario with 40 bps spread, you would loose about 30% monthly, which would leave you with almost exactly zero profit.

With wider spread such as 100bps, you can achieve better profitability of about +10% monthly at the cost of less rewards. This seems much better, but note that this backtest is far from a complete implementation of hummingbot trading logic and while it provides a good idea about the profitability, it is not very realistic.

You can add more logic to your quoting strategy in order to increase profitability or shrink spread and increase rewards. I also suggest you improve this notebook and open a pull request on github to share it with the community. If you do so, I will check your code for mistakes and might provide tips for further improvement.

### Follow ups

- how are the results stable in time? how to stabilize them?
- how much rewards would you get depending on spread? which spread maximizes trading profits + rewards?
- how does inventory balancing affect profitability? can we simulate it?
- how does the lag between quoting and fill affect profitability? can we simulate it?